<h1>Acromelic Frontonasal Dysostosis (ZSWIM6): Twigg et al 2016</h1>
<p>Extract the clinical data from <a href="https://pubmed.ncbi.nlm.nih.gov/26706854/"target="__blank">Twigg SR, et al. (2016) Acromelic frontonasal dysostosis and ZSWIM6 mutation: phenotypic spectrum and mosaicism. Clin Genet. 90(3):270-5. PMID:26706854</a>.<p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
import math
from csv import DictReader
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import re
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import *
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.6.3


In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
pmid="PMID:26706854"
title="Acromelic frontonasal dysostosis and ZSWIM6 mutation: phenotypic spectrum and mosaicism"
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", pmid=pmid, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)

In [3]:
df = pd.read_csv("input/twigg_2016.csv", delimiter="\t")

In [4]:
df

,subject,gender,eyes,nose,mouth,skull,morphology,development
0,1-1,F,Hypertelorism,"Wide nasal bridge, short nasal ridge, bifid nasal tip",Normal,Normal,Normal,Normal
1,1-2,M,"Severe hypertelorism, downslanting palpebral fissures","Wide nasal bridge, widely spaced nasal alae, widely separated slit‐like nares","Carp‐shaped mouth, midline notch in upper lip, cleft palate","Bony defect of anterior cranial fossa, parietal foramina","Interhemispheric lipoma, partial agenesis of the corpus callosum",Severe motor and neurocognitive delay
2,2,M,"Hypertelorism, bilateral ptosis, downslanting palpebral fissures, bilateral cataract","Wide nasal bridge, short nasal ridge, bifid nasal tip, widely spaced nasal alae, widely separated slit‐like nares","Carp‐shaped mouth, long philtrum, midline notch in upper lip, cleft palate",Bony defect of anterior cranial fossa,Anterior interhemispheric lipoma,"Severe psychomotor delay, absence of speech, does not walk aged 8 years"
3,3,F,"Hypertelorism, downslanting palpebral fissures","Aplasia/hypoplasia of the nasal bones, wide nasal bridge, bifid nasal tip",Midline notch in upper lip,-,-,-


In [5]:
column_mapper_d = defaultdict(ColumnMapper)

In [6]:
eyeMapper = CustomColumnMapper(concept_recognizer=hpo_cr)
eyeMapper.preview_column(df['eyes'])
# Add the mapper to the dictionary, using the label of the column in the Supplemental Table
column_mapper_d['eyes'] = eyeMapper

In [7]:
nose_d = {"widely spaced nasal alae": "Bifid nose"
}
noseMapper = CustomColumnMapper(concept_recognizer=hpo_cr, custom_map_d=nose_d)
noseMapper.preview_column(df['nose'])
column_mapper_d['nose'] = noseMapper

In [8]:
mouth_d = { "midline notch in upper lip": "Microform cleft of the upper lip"}
# TODO Add dictionary when latest terms there , custom_map_d=mouth_d
mouthMapper = CustomColumnMapper(concept_recognizer=hpo_cr)
mouthMapper.preview_column(df['mouth'])
column_mapper_d['mouth'] = mouthMapper

In [9]:
skull_d = {"Bony defect of anterior cranial fossa": "Calvarial skull defect"}
skullMapper = CustomColumnMapper(concept_recognizer=hpo_cr, custom_map_d=skull_d)
skullMapper.preview_column(df['skull'])
column_mapper_d['skull'] = skullMapper

In [10]:
morphology_d = {"interhemispheric lipoma": "Midline central nervous system lipomas"}
morphologyMapper = CustomColumnMapper(concept_recognizer=hpo_cr, custom_map_d=morphology_d)
morphologyMapper.preview_column(df['morphology'])
column_mapper_d['morphology']=morphologyMapper

In [11]:
dev_d = {"absence of speech": "Absent speech",
        "does not walk aged 8 years": "Delayed ability to walk"}
developmentMapper  = CustomColumnMapper(concept_recognizer=hpo_cr, custom_map_d=dev_d)
developmentMapper.preview_column(df['development'])
column_mapper_d['development'] = developmentMapper

<h2>Variants</h2>
<p>All individuals were found to have the ZSWIM6 c.3487C>T; p.Arg1163Trp variant.</p>

In [12]:
validator = VariantValidator(genome_build='hg38')
transcript = "NM_020928.2"
var_hgvs = "c.3487C>T"
v = validator.encode_hgvs(hgvs=var_hgvs, custom_transcript=transcript)
v.set_heterozygous()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_020928.2%3Ac.3487C>T/NM_020928.2?content-type=application%2Fjson


In [13]:
v_d = {"c.3487C>T":v}
df['variant'] = "c.3487C>T"
varMapper = VariantColumnMapper(variant_d=v_d, variant_column_name="variant")

In [14]:
sexMapper = SexColumnMapper(male_symbol='M', female_symbol='F', column_name='gender')
sexMapper.preview_column(df['gender'])

individual_column_name = 'subject'


encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name=individual_column_name,
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata,
                        pmid=pmid)


disease_id = 'OMIM:618443'
disease_name = 'Neurodevelopmental disorder with or without variable brain abnormalities'
individual_list = []
disease_label = "Acromelic frontonasal dysostosis"
disease_id = "OMIM:603671"
encoder.set_disease(disease_id=disease_id, label=disease_label)

In [15]:
individuals = encoder.get_individuals()    
print(f"Created {len(individuals)} individual objects")

Created 4 individual objects


In [16]:
ppacket_list = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=ppacket_list)
from IPython.display import display, HTML
display(HTML(table.to_html()))

In [17]:
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              pmid=pmid,
                                              metadata=metadata.to_ga4gh())

We output 4 GA4GH phenopackets to the directory phenopackets
